# TRABALHO FINAL DE MATEMÁTICA DISCRETA

## INTRODUÇÃO:

<<<<<< INTRO >>>>>>

## EXTRAINDO OS DADOS:

Conseguir os dados para executar o projeto foi de fato complicado. Procuranmos em sites como o [OpenFlights](https://openflights.org/#), que realmente possuem dados sobre as inúmeras linhas aéreas existentes no planeta, porpem o grande problema era extrair os dados todos de uma vez, não existia um mecanismo que possibilitasse o download de todos os dados de uma só vez. 

A solução encontrada foi uma base de dados presente no site [Kaggle](https://www.kaggle.com/datasets/open-flights/flight-route-database) com dados sobre linhas aéreas entre mais de 3 mil aeroportos em diversos países em janeiro de 2012. Obviamente, desde de então, novos aeroportos foram criados e, por consequência, novas conexões. Contudo, visto que o objetivo do trabalho é aplicar a teoria vista em aula para problemas da vida real, problemas práticos, acredito que a desatualização da base de dados não seja um problema.

Um vez com os dados em mãos, teríamos que tratá-los para que pudessem atender as nossas necessidades. Segue abaixo o código usado para tratar a base de dados inicial e gerar duas bases de dados derivadas: uma apenas com aeroportos do brasileiros e outra com todos. Sobre o tratamento, um comentário relrevante é de que usamos, além da bibliotaca `pandas`, a biblioteca de python `airportsdata` para traduzir os dados, visto que a base inicial possui apenas o código IATA (International Air Transport Association) dos aeroportos e iriamos precisar de outros dados, como latitude e longitude para calcular as distâncias entre eles, por exemplo. 

In [61]:
import pandas as pd  
import airportsdata

#lendo o dataframe
dataframe_bruto = pd.read_csv("database\\routes.csv")

#carregando dados da biblioteca
dicionario_dos_aeroportos = airportsdata.load('IATA')

#dados que vamos extrai da biblioteca que não existem no dataframe inicial
lista_de_colunas = ["route_id", "source_airport_icao", "source_airport_name", "source_airport_city", "source_airport_country", "source_airport_elevation",
                     "source_airport_lat", "source_airport_lon", "destination_airport_icao", "destination_airport_name", "destination_airport_city", 
                     "destination_airport_country", "destination_airport_elevation", "destination_airport_lat", "destination_airport_lon"]

lista_de_origens = list(dataframe_bruto[" source airport"])
lista_de_destinos = list(dataframe_bruto[" destination apirport"])
lista_de_dados = list()

#extraindo dados da biblioteca
for cada_origem, cada_destino in zip(lista_de_origens, lista_de_destinos):
    try:
        lista_auxiliar = list()
        lista_auxiliar.append(f"{cada_origem}-{cada_destino}")
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_origem]["icao"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_origem]["name"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_origem]["city"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_origem]["country"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_origem]["elevation"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_origem]["lat"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_origem]["lon"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_destino]["icao"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_destino]["name"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_destino]["city"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_destino]["country"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_destino]["elevation"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_destino]["lat"])
        lista_auxiliar.append(dicionario_dos_aeroportos[cada_destino]["lon"])
        lista_de_dados.append(lista_auxiliar)
    except:
        lista_de_dados.append(list())

dataframe_filtrado = pd.DataFrame(data=lista_de_dados, columns=lista_de_colunas)
dataframe_bruto["route_id"] = dataframe_filtrado["route_id"]

#criando o dataframe final
dataframe_final = pd.merge(dataframe_bruto, dataframe_filtrado, on='route_id', how='inner')
dataframe_final = dataframe_final.dropna(thresh=2)
dataframe_final = dataframe_final.groupby("route_id", as_index=False).first().reset_index(drop=True)

#filtrando para aeroportos brasileiros
dataframe_final_brasil = dataframe_final[dataframe_final["source_airport_country"] == "BR"].reset_index(drop=True)

""" TRECHO PARA SALVAR AS NOVAS BASES DE DADOS
dataframe_final.to_csv("database\\dataframe_final_mundo.csv")
dataframe_final_brasil.to_csv("database\\dataframe_final_brasil.csv")
"""

print(dataframe_final.shape)

(36800, 24)


## CÁLCULO DE ROTAS:

Como foi exposto do documento enviado com os objetivos do trabalho, um dos nossos objetivos é desenvolver um programa, que tem como base o algorítimo de Dijskra, que é usado para encontrar o menor caminho entre dois vértices em um gráfico dirigido, para encontrar as rotas aéreas mais eficientes com base em horas de voos. Obviamente, entre dois vértices seria extremamente simples, por isso a nossa aplicação vai encontrar a rota mais enficiênte em viagens complexas, que possuem multiplas paradas, paradas essas que devem estar ordenadas. 

No nosso modelo, os aeroportos seriam os vértices e as rotas aéreas as arestas. Porém, na nossa base não temos informações sobre as durações dos voos, mas como alternativa vamos utilizar de latitude e longitude para calcular a distância aproximada entre eles e, após isso, calcular a duração da viagem para um avião comercial médio. Segue abaixo o código utilizado para calcular a distância entre os aeroportos, levando em consideração uma curvatura uniforme da Terra, e a duração aproximada dessas viagens. 

In [62]:
import math

def cal_dist(lat_inicial:float, lon_inicial:float, lat_final:float, lon_final:float) -> float:
    """
    """
    try:
        dif_lat = math.radians(float(lat_final) - float(lat_inicial))
        dif_lon = math.radians(float(lon_final) - float(lon_inicial))

        raio_terra_km = 6370

        #funções de haversine para distância esférica
        hav_lat = math.sin(dif_lat/2)**2
        hav_lon = math.sin(dif_lon/2)**2

        cos_1 = math.cos(float(math.radians(lat_inicial)))
        cos_2 = math.cos(float(math.radians(lat_final)))

        dist_real = 2*raio_terra_km*math.asin(math.sqrt(hav_lat + cos_1 * cos_2 * hav_lon))

        return round(dist_real, 3)
    
    except:
        return None


def cal_horas(dist_km:float) -> float:
    """
    """
    try:
        velociade_km_h = 850
        tempo_horas = round(float(dist_km)/velociade_km_h, 3)

        return tempo_horas
    
    except:
        return None

Definidas as funções, vamos agora aplicá-las linha a linha e salvar no nosso dataframe a distância entre os aeroportos em questão, ou seja, o comprimento da rota. Além disso, vamos adicionar ao dataframe a duração estimada da viagem com base na velociade de um avião comercial médio, como foi dito anteriormente, e no formato decimal, visto que só usaremos esses dados para fazer comparações e operações matemáticas relacionadas à aplicação do algorítimo de Disjkra. 

In [63]:
lista_lat_inicial = list(dataframe_final["source_airport_lat"])
lista_lon_inicial = list(dataframe_final["source_airport_lon"])
lista_lat_final = list(dataframe_final["destination_airport_lat"])
lista_lon_final = list(dataframe_final["destination_airport_lon"])

lista_distancias = list()
lista_tempo_de_viagem = list()
for index in range(0, len(lista_lat_inicial)):

    #extraindo os dados
    lat_inicial = lista_lat_inicial[index]
    lon_inicial = lista_lon_inicial[index]
    lat_final = lista_lat_final[index]
    lon_final = lista_lon_final[index]

    #claculando a distância e o tempo
    dist = cal_dist(lat_inicial, lon_inicial, lat_final, lon_final)
    tempo = cal_horas(dist)

    lista_distancias.append(dist)
    lista_tempo_de_viagem.append(tempo)

#criando as colunas com os novos dados 
dataframe_final["distancia"] = lista_distancias
dataframe_final["tempo_de_viagem"] = lista_tempo_de_viagem

#excluindo linhas, que por algum problema, não foi possível calcular ou a distância, ou o tempo de viagem
dataframe_final = dataframe_final.dropna(thresh=2).reset_index(drop=True)

print(dataframe_final.shape)

(36800, 26)


Após inserir na nossa base de dados as informações necessárias, basta agora criar o programa que recebe nome de cidades em ordem, cidades essas que devem ter aeroportos, e retorna uma espécie de plano de voo buscando a menor rota entre cada aeroporto. 